#INPUT FILES

The VWF NNK experiment was split into three different runs and the generated high throughput sequencing files have been deposited to the NCBI Sequence Read Archive (project accession number #PRJNA356764). #PRJNA296481 encompasses 3 sets of 50bp paired-end .fastq files used in the following pipelines:

- Set 1: #SRR5097080
- Set 2: #SRR5097081
- Set 3: #SRR5097082

##Pipeline for set 1

###*clean_fastq_ORF.py*
    - seed match to position the read
    - sense-antisense read match at NNK
    - minimum read quality of 5 on the scale of 0-41

In [ ]:
#!/usr/bin/env python
# k2rt 05-11-2013
# 
#import possible modules the script might need
import sys
import re
import mod575

# read in the file numb
number = sys.argv[1]
#open both fastq files
fileName1=sys.argv[2]
fileName2=sys.argv[3]
file1=open(fileName1,'r')
file2=open(fileName2,'r')
#open output file for peptides
fileName5=str(number)+"_sense"
fileName6=str(number)+"_antisense"
outfile3=open(fileName5,'w')
outfile4=open(fileName6,'w')
#create a little stats file over all the reads to know when we are throwing reads out
stats=open('stats_clean_fastq.txt','a')
#some counters to keep track of the running file and collect numbers for stats
counter=0
good_reads=0
unmatched_pairs=0
seed1=0
seed2=0
seed3=0
seed4=0
seed_unmatched=0
bad_quality=0
sense_antisense_unmatched=0
OOF=0

#define a generator to return 4 lines from each paired read file
def get_read_set(file1, file2):
	'''Expects to open files as input. Returns 8 lines of data, four from
each file. Return characters are stripped from each line.
'''
	while True:
		line1 = file1.readline().strip()
		line2 = file1.readline().strip()
		line3 = file1.readline().strip()
		line4 = file1.readline().strip()
		line5 = file2.readline().strip()
		line6 = file2.readline().strip()
		line7 = file2.readline().strip()
		line8 = file2.readline().strip()
		if (line1 and line2 and line3 and line4 and line5 and line6 and line7 and line8):
			yield line1, line2, line3, line4, line5, line6, line7, line8
		else:
			break

#start going through the fastq files 8 lines at a time using the generator
for pairedRead in get_read_set(file1, file2):
	counter+=1
	row1=pairedRead[0]
	row2=pairedRead[1]
	row3=pairedRead[2]
	row4=pairedRead[3]
	row5=pairedRead[4]
	row6=pairedRead[5]
	row7=pairedRead[6]
	row8=pairedRead[7]
#check if the @header matches between paired fastqfiles, if not count the error and continue
	head1=row1.split()
	head2=row5.split()
	if not head1[0]==head2[0]:
		unmatched_pairs+=1
		continue
#check both pairs of reads for matching of at least one of three seeds of 8bp. If no match, goes to junk

	if re.match(r'^[ATCGN]{12}GAGCAGG[ATCGN]+',row2):
		sense=row2[27:45]
		antisense=row6[29:47]
		sense_quality=row4[27:45]
		antisense_quality=row8[29:49]
		sense_row=row2
		antisense_row=row6
		seed1+=1
	elif re.match(r'^[ATCGN]{19}CGCCCAAC[ATCGN]+',row2):
		sense=row2[27:45]
		antisense=row6[29:47]
		sense_quality=row4[27:45]
		antisense_quality=row8[29:49]
		sense_row=row2
		antisense_row=row6
		seed2+=1
	elif re.match(r'^[ATCGN]{11}ATCAGAGGC[ATCGN]+',row2):
		sense=row6[27:45]
		antisense=row2[29:47]
		sense_quality=row8[27:45]
		antisense_quality=row4[29:49]
		sense_row=row6
		antisense_row=row2
		seed3+=1
	elif re.match(r'^[ATCGN]{20}AGGATTTCC[ATCGN]+',row2):
		sense=row6[27:45]
		antisense=row2[29:47]
		sense_quality=row8[27:45]
		antisense_quality=row4[29:49]
		sense_row=row6
		antisense_row=row2
		seed4+=1
	else:
		if re.match(r'^[ATCGN]{12}GAGCAGG[ATCGN]+',row6):
			sense=row6[27:45]
			antisense=row2[29:47]
			sense_quality=row8[27:45]
			antisense_quality=row4[29:49]
			sense_row=row6
			antisense_row=row2
			seed1+=1
		elif re.match(r'^[ATCGN]{19}CGCCCAAC[ATCGN]+',row6):
			sense=row6[27:45]
			antisense=row2[29:47]
			sense_quality=row8[27:45]
			antisense_quality=row4[29:49]
			sense_row=row6
			antisense_row=row2
			seed2+=1
		elif re.match(r'^[ATCGN]{11}ATCAGAGGC[ATCGN]+',row6):
			sense=row2[27:45]
			antisense=row6[29:47]
			sense_quality=row4[27:45]
			antisense_quality=row8[29:49]
			sense_row=row2
			antisense_row=row6
			seed3+=1
		elif re.match(r'^[ATCGN]{20}AGGATTTCC[ATCGN]+',row6):
			sense=row2[27:45]
			antisense=row6[29:47]
			sense_quality=row4[27:45]
			antisense_quality=row8[29:49]
			sense_row=row2
			antisense_row=row6
			seed4+=1	
		else:
			seed_unmatched+=1
			continue

	#checking for matching
	if not sense==mod575.reverse_compliment(antisense):
		sense_antisense_unmatched+=1
		continue


#checking for quality. translating the quality code from ASCII33 and removing any read where the pos quality is lower than 5 on scale from 0-41!
	status=0
	for pos in sense_quality:
		score=int(ord(str(pos)))-33
		if score < 5:
			status+=1
	for pos in antisense_quality:
		score=int(ord(str(pos)))-33
		if score < 5:
			status+=1
	if status !=0:
		bad_quality+=1
		continue		

	good_reads+=1
	print >> outfile3, sense_row
	print >> outfile4, antisense_row
	continue		


print >> stats,number,counter,good_reads,unmatched_pairs,seed1,seed2,seed3,seed4,seed_unmatched,sense_antisense_unmatched,bad_quality

file1.close()
file2.close()
outfile3.close()
outfile4.close()
stats.close()

STATS (72976223/84083495 => ~86.8%):

NNK 84083495 72976223 0 78209684 246784 941149 5519 4680359 6113876 313037

###*barcode.py*:
    - reads in the filtered sense/antisense reads
    - seperates the reads by 6 different barcodes

In set 1 , barcodes 1 and 2 are controls, 3 and 4 are ADAMTS13 treated libraries

In [ ]:
#!/usr/bin/env python
# k2rt 05-11-2013
# 
#import possible modules the script might need
import sys
import re
import mod575

#open both fastq files
fileName1=sys.argv[1]
fileName2=sys.argv[2]
file1=open(fileName1,'r')
file2=open(fileName2,'r')
#open output file for peptides
outfile1=open('1_sense','w')
outfile2=open('2_sense','w')
outfile3=open('3_sense','w')
outfile4=open('4_sense','w')
outfile5=open('5_sense','w')
outfile6=open('6_sense','w')

#create a little stats file over all the reads to know when we are throwing reads out
stats=open('stats_barcode.txt','a')
#some counters to keep track of the running file and collect numbers for stats
counter=0
good_reads=0
seed1=0
seed2=0
seed3=0
seed4=0
seed5=0
seed6=0

seed_unmatched=0


#define a generator to return 4 lines from each paired read file
def get_read_set(file1, file2):
	'''Expects to open files as input. Returns 1 line of data, 1 from
each file. Return characters are stripped from each line.
'''
	while True:
		line1 = file1.readline().strip()
		line2 = file2.readline().strip()

		if (line1 and line2):
			yield line1, line2
		else:
			break

#start going through the fastq files 8 lines at a time using the generator
for pairedRead in get_read_set(file1, file2):
	counter+=1
	row1=pairedRead[0]
	row2=pairedRead[1]

#check both pairs of reads for matching of at least one of three seeds of 8bp. If no match, goes to junk

	if re.match(r'^[ATCGN]{7}GAATC[ATCGN]+',row1):
		seed1+=1
		print >> outfile1, row1
		
	elif re.match(r'^[ATCGN]{6}GTCTA[ATCGN]+',row2):
		seed1+=1
		print >> outfile1, row1
		
	elif re.match(r'^[ATCGN]{7}TCTGA[ATCGN]+',row1):
		seed2+=1
		print >> outfile2, row1
		
	elif re.match(r'^[ATCGN]{6}CAGAG[ATCGN]+',row2):
		seed2+=1
		print >> outfile2, row1
		
	elif re.match(r'^[ATCGN]{7}CAGTC[ATCGN]+',row1):
		seed5+=1
		print >> outfile5, row1
		
	elif re.match(r'^[ATCGN]{6}TTCAC[ATCGN]+',row2):
		seed5+=1
		print >> outfile5, row1
		
	elif re.match(r'^[ATCGN]{7}AGTCT[ATCGN]+',row1):
		seed6+=1
		print >> outfile6, row1
	
	elif re.match(r'^[ATCGN]{6}AGTCT[ATCGN]+',row2):
		seed6+=1
		print >> outfile6, row1
	
	elif re.match(r'^[ATCGN]{7}TGCAA[ATCGN]+',row1):
		seed3+=1
		print >> outfile3, row1
	
	elif re.match(r'^[ATCGN]{6}TCAGG[ATCGN]+',row2):
		seed3+=1
		print >> outfile3, row1
		
	elif re.match(r'^[ATCGN]{7}ACAGT[ATCGN]+',row1):
		seed4+=1
		print >> outfile4, row1
	
	elif re.match(r'^[ATCGN]{6}ATGTC[ATCGN]+',row2):
		seed4+=1
		print >> outfile4, row1
	
	else:
		seed_unmatched+=1

print >> stats,counter,seed1,seed2,seed3,seed4,seed5,seed6,seed_unmatched

file1.close()
file2.close()
outfile1.close()
outfile2.close()
outfile3.close()
outfile4.close()
outfile5.close()
outfile6.close()
stats.close()

STATS:

all barcode1 barcode2 barcode3 barcode4 barcode5 barcode6 nomatch
72976223 18164303 18103794 18723277 17976287 2374 3526 2662


###*count_peptides.py*:
    - counts the unique peptides
    - counts the unique DNA sequencing encoding that peptide
    - sorts peptides by count

In [ ]:
#!/usr/bin/env python
# k2rt 08-10-2012
# this script recognizes specifically the sense strand of Colin's data

# import possible modules the script might need
import sys
import re
import mod575
from collections import defaultdict

# read in the file numb
number = sys.argv[1]
fileName1=number+'_sense'
fileName2=number+'_peptide'
input=open(fileName1,'r')
outfile=open(fileName2,'w')

peptides=dict()
sequences=defaultdict(defaultdict)
counter=0

for line in input:
	counter+=1
	line=line.strip()
	sense=line[27:45]
	peptide=mod575.translate_dna(sense)
	if peptide in peptides:
		peptides[peptide]+=1
	else:
		peptides[peptide]=1

	if peptide in sequences:
		if sense in sequences[peptide]:
			sequences[peptide][sense]+=1
		else:
			sequences[peptide][sense]=1
	else:
		sequences[peptide][sense]=1

for peptide in sorted(peptides, key=peptides.get, reverse=True):
	seqlist=[]	
	for sense in sequences[peptide]:
		seqdata=str(sense)+':'+str(sequences[peptide][sense])
		seqlist.append(seqdata)
	seqs=';'.join(seqlist)
	data=peptide+'\t'+str(peptides[peptide])+'\t'+seqs
	print >> outfile, data

input.close()
outfile.close()

In [ ]:
#grep command to remove peptides containing stop codon
grep -v 'X' 1_peptide > 1_noX_peptide
etc.


|Treatment|Seq_name|Reads|Percent|Pep_name|Peptide_count|Peptide_noX|
|-|-|-|-|-|-|-|
|Control1_set1|1_sense|18,164,303|24.9%|1_peptide|529,198|486,090|
|Control2_set1|2_sense|18,103,794|24.8%|2_peptide|518,022|476,329|
|ATS1_set1|3_sense|18,723,277|25.7%|3_peptide|570,009|521,164|
|ATS2_set1|4_sense|17,976,287|24.6%|4_peptide|620,543|563,065|


##Pipeline for set 2

###*clean_fastq_ORF.py*:
    - Used the same script as above

STATS:

Lane1 95835969 58240300 0 61496916 310103 717407 9280 33302263 4074659 218747

###*barcode.py*:
    - Used the same script as above

STATS:
all **barcode1 barcode2** barcode3 barcode4 **barcode5** barcode6 nomatch
58240300 **17795244 20224913** 74473 120151 **20020332** 4291 896

In set 2 , barcodes 1 and 2 are controls and 5 is the ADAMTS13 treated library

###*count_peptides.py*:
    - Used the same script as above


|Treatment|Seq_name|Reads|Percent|Pep_name|Peptide_count|Peptide_noX|
|-|-|-|-|-|-|-|
|Control1_set2|1_sense|17795244|30.6%|1_peptide|500,808|459,596|
|Control2_set2|2_sense|20224913|34.7%|2_peptide|520,348|476,524|
|ATS1_set2|5_sense|20020332|34.4%|5_peptide|697,037|624,528|



##Pipeline for set 3

###*clean_fastq_ORF.py*:
    - Used the same script as above

STATS:

Lane2 90262326 54627695 0 57861693 200158 614395 7386 31578694 3889895 166042


###*barcode.py*:
    - Used the same script as above

STATS:
all barcode1 barcode2** barcode3 barcode4 **barcode5 barcode6 nomatch
54627695 1736868 2242992 **19193241 23267645** 6189067 1992756 5126

In set 3 , barcodes 3 and 4 are ADAMTS13 treated libraries

###*count_peptides.py*:
    - Used the same script as above


|Treatment|Seq_name|Reads|Percent|Pep_name|Peptide_count|Peptide_noX|
|-|-|-|-|-|-|-|
|ATS1_set3|3_sense|19193241|35.1%|3_peptide|562,951|512,064|
|ATS1_set3|4_sense|23267645|42.6%|4_peptide|673,966|606,714|


##Merge all above peptide counts for DESEQ2

*merge_9.py*:
    - merges petides with at least 4 counts over all the samples

In [ ]:
#!/usr/bin/env python
# k2rt 08-10-2012
# this script recognizes specifically the sense strand of Colin's data

# import possible modules the script might need
import sys
import re

# read in the file numb and create the input output files
treatment = sys.argv[1]
fileName1=sys.argv[2]
fileName2=sys.argv[3]
fileName3=sys.argv[4]
fileName4=sys.argv[5]
fileName5=sys.argv[6]
fileName6=sys.argv[7]
fileName7=sys.argv[8]
fileName8=sys.argv[9]
fileName9=sys.argv[10]

sumlimit=int(sys.argv[11])
input1=open(fileName1,'r')
input2=open(fileName2,'r')
input3=open(fileName3,'r')
input4=open(fileName4,'r')
input5=open(fileName5,'r')
input6=open(fileName6,'r')
input7=open(fileName7,'r')
input8=open(fileName8,'r')
input9=open(fileName9,'r')

outfile=open('DESeq/'+treatment+'_merged','w')
#make up necessary dictionaries
peptides=dict()
pep1=dict()
pep2=dict()
pep3=dict()
pep4=dict()
pep5=dict()
pep6=dict()
pep7=dict()
pep8=dict()
pep9=dict()

#read in the peptides in the two files

print >> outfile, 'peptide\tuntreated1\tuntreated2\tuntreated3\tuntreated4\ttreated1\ttreated2\ttreated3\ttreated4\ttreated5'

for line in input1:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		print 'sth weird'
	else:
		peptides[row[0]]=1
	if row[0] in pep1:
		print 'sth weird'
	else:
		pep1[row[0]]=row[1]
for line in input2:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep2:
		print 'sth weird'
	else:
		pep2[row[0]]=row[1]

for line in input3:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep3:
		print 'sth weird'
	else:
		pep3[row[0]]=row[1]
for line in input4:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep4:
		print 'sth weird'
	else:
		pep4[row[0]]=row[1]
for line in input5:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep5:
		print 'sth weird'
	else:
		pep5[row[0]]=row[1]

for line in input6:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep6:
		print 'sth weird'
	else:
		pep6[row[0]]=row[1]

for line in input7:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep7:
		print 'sth weird'
	else:
		pep7[row[0]]=row[1]

for line in input8:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep8:
		print 'sth weird'
	else:
		pep8[row[0]]=row[1]

for line in input9:
	line=line.strip()
	row=line.split()
	if row[0] in peptides:
		peptides[row[0]]+=1
	else:
		peptides[row[0]]=1
	if row[0] in pep9:
		print 'sth weird'
	else:
		pep9[row[0]]=row[1]

for peptide in peptides:
	sum=0
	out=list()
	out.append(peptide)
	if peptide in pep1:
		out.append(pep1[peptide])
		count_pep1=int(pep1[peptide])
	else:
		out.append('0')
		count_pep1=0
	if peptide in pep2:
		out.append(pep2[peptide])
		count_pep2=int(pep2[peptide])
	else:
		out.append('0')
		count_pep2=0
	if peptide in pep3:
		out.append(pep3[peptide])
		count_pep3=int(pep3[peptide])
	else:
		out.append('0')
		count_pep3=0
	if peptide in pep4:
		out.append(pep4[peptide])
		count_pep4=int(pep4[peptide])
	else:
		out.append('0')
		count_pep4=0
	if peptide in pep5:
		out.append(pep5[peptide])
		count_pep5=int(pep5[peptide])
	else:
		out.append('0')
		count_pep5=0

	if peptide in pep6:
		out.append(pep6[peptide])
		count_pep6=int(pep6[peptide])
	else:
		out.append('0')
		count_pep6=0

	if peptide in pep7:
		out.append(pep7[peptide])
		count_pep7=int(pep7[peptide])
	else:
		out.append('0')
		count_pep7=0

	if peptide in pep8:
		out.append(pep8[peptide])
		count_pep8=int(pep8[peptide])
	else:
		out.append('0')
		count_pep8=0

	if peptide in pep9:
		out.append(pep9[peptide])
		count_pep9=int(pep9[peptide])
	else:
		out.append('0')
		count_pep9=0

	out='\t'.join(out)
	sum=count_pep1+count_pep2+count_pep3+count_pep4+count_pep5+count_pep6+count_pep7+count_pep8+count_pep9

	if sum<sumlimit:
		continue
	else:
		print >> outfile, out

input1.close()
input2.close()
input3.close()
input4.close()
outfile.close()

###DESEQ2

The merged file is used as an input for R based package DESeq2

In [ ]:
#DESEQ2

library(DESeq2)
data=read.table('ALL_VWF73_4_merged',header=TRUE,row.names=1)
countData=as.matrix(data)
colData=data.frame(condition=factor(c("control","control","control","control","treatement","treatement","treatement","treatement","treatement")))
dds=DESeqDataSetFromMatrix(countData, colData, formula(~ condition))

dds=DESeq(dds)
res=results(dds)

pdf('ALL_VWF73_4_deseq2_plotMA.pdf')
plotMA(dds,alpha=.05,ylim=c(-2,2),main='ALL_ADA_4')
dev.off()

 write.table(res,file="ALL_VWF73_4_deseq2.txt", col.names=FALSE, append=FALSE, quote=FALSE)

All reads with adjusted p-value <=0.05 is considered significantly enriched/depleted

In [ ]:
#to extract significant peptides from the DESEQ2 output file
awk '$3>0' ALL_VWF73_4_deseq2.txt | awk '$7<=0.05' | sort -g -k 7 > enriched_top.txt
awk '$3<0' ALL_VWF73_4_deseq2.txt | awk '$7<=0.05' | sort -g -k 7 > depleted_top.txt
